In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [26]:
df = pd.read_csv('feature_output_10k_final.csv')

In [27]:
df

,Unnamed: 0.1,Unnamed: 0,text,generated,length,label,neg,neu,pos,compound,subjectivity_score,flesch_reading_ease_score,flesch_kincaid_grade,Perplexity Score,split_text,no_discourse_markers,no_pronouns,mean_words_per_sentence,grammatical_errors,named_entity_counts
0,0,321478,Becoming your on self is a crucial aspect of a...,1.0,2276,AI,0.038,0.683,0.279,0.9987,0.590000,57.91,10.6,9.079517,"['Becoming', 'your', 'on', 'self', 'is', 'a', ...",27,21,20.315789,60,1
1,1,49275,Phones and Driving\n\nPhones and techknowledge...,0.0,2381,Human,0.092,0.819,0.090,-0.5986,0.534348,81.33,5.7,33.769810,"['Phones', 'and', 'Driving\n\nPhones', 'and', ...",29,11,14.419355,69,4
2,2,123215,"You should be a Seagoing Cowboy.\n\nIt is fun,...",0.0,827,Human,0.018,0.755,0.227,0.9894,0.455934,95.67,2.3,21.495440,"['You', 'should', 'be', 'a', 'Seagoing', 'Cowb...",6,3,8.222222,28,4
3,3,190730,"As an 8th grader, I believe that committing to...",1.0,2325,AI,0.006,0.811,0.184,0.9956,0.543547,57.10,10.9,7.064874,"['As', 'an', '8th', 'grader,', 'I', 'believe',...",20,28,22.222222,54,2
4,4,329038,"You should join the Seagoing Cowboys because, ...",0.0,1183,Human,0.034,0.789,0.177,0.9886,0.514757,85.28,4.2,50.013187,"['You', 'should', 'join', 'the', 'Seagoing', '...",16,10,10.666667,25,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,188710,Seeking advice from multiple people when facin...,1.0,1987,AI,0.051,0.785,0.164,0.9892,0.482265,40.58,13.1,34.399616,"['Seeking', 'advice', 'from', 'multiple', 'peo...",12,10,20.466667,28,2
9996,9996,9688,"The Face on Mars, is it really what NASA says ...",0.0,2374,Human,0.023,0.938,0.039,0.8044,0.443304,74.22,8.4,42.247635,"['The', 'Face', 'on', 'Mars,', 'is', 'it', 're...",20,22,19.954545,40,10
9997,9997,31949,"Title: Exploring Venus\n\nVenus, the second pl...",1.0,2470,AI,0.022,0.926,0.052,0.9022,0.494501,48.03,12.3,17.237017,"['Title:', 'Exploring', 'Venus\n\nVenus,', 'th...",21,6,21.444444,59,9
9998,9998,481082,"Hey, Mrs. Smith! Here's my essay on the benefi...",1.0,2028,AI,0.064,0.719,0.217,0.9968,0.457093,77.10,9.4,9.456690,"['Hey,', 'Mrs.', 'Smith!', ""Here's"", 'my', 'es...",25,22,24.125000,61,5


In [28]:
features_to_divide = ['no_discourse_markers','no_pronouns','grammatical_errors','named_entity_counts']
df[features_to_divide] = df[features_to_divide].div(df['length'],axis=0)

In [29]:
columns_to_drop = [0, 1, 2, 4, 5, 14] 
df = df.drop(columns=df.columns[columns_to_drop])
df

,generated,neg,neu,pos,compound,subjectivity_score,flesch_reading_ease_score,flesch_kincaid_grade,Perplexity Score,no_discourse_markers,no_pronouns,mean_words_per_sentence,grammatical_errors,named_entity_counts
0,1.0,0.038,0.683,0.279,0.9987,0.590000,57.91,10.6,9.079517,0.011863,0.009227,20.315789,0.026362,0.000439
1,0.0,0.092,0.819,0.090,-0.5986,0.534348,81.33,5.7,33.769810,0.012180,0.004620,14.419355,0.028979,0.001680
2,0.0,0.018,0.755,0.227,0.9894,0.455934,95.67,2.3,21.495440,0.007255,0.003628,8.222222,0.033857,0.004837
3,1.0,0.006,0.811,0.184,0.9956,0.543547,57.10,10.9,7.064874,0.008602,0.012043,22.222222,0.023226,0.000860
4,0.0,0.034,0.789,0.177,0.9886,0.514757,85.28,4.2,50.013187,0.013525,0.008453,10.666667,0.021133,0.003381
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1.0,0.051,0.785,0.164,0.9892,0.482265,40.58,13.1,34.399616,0.006039,0.005033,20.466667,0.014092,0.001007
9996,0.0,0.023,0.938,0.039,0.8044,0.443304,74.22,8.4,42.247635,0.008425,0.009267,19.954545,0.016849,0.004212
9997,1.0,0.022,0.926,0.052,0.9022,0.494501,48.03,12.3,17.237017,0.008502,0.002429,21.444444,0.023887,0.003644
9998,1.0,0.064,0.719,0.217,0.9968,0.457093,77.10,9.4,9.456690,0.012327,0.010848,24.125000,0.030079,0.002465


In [30]:
X = df.drop(columns = ['generated'])
y = df.generated

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=28)

In [31]:
# SVM_model = SVC(kernel='linear', random_state=28)

# param_grid = {
#     'C': [0.1, 1, 10, 100, 150], 
# }

# grid_search = GridSearchCV(estimator=SVM_model, param_grid=param_grid, cv=5)
# grid_search.fit(X_train, y_train)
# best_params = grid_search.best_params_
# print("Best Parameters:", best_params)

# best_SVM_model = grid_search.best_estimator_
# y_pred = best_SVM_model.predict(X_test)

Best Parameters: {'C': 100}

In [32]:
SVM_model = SVC(
    kernel='linear',
    C = 100,
    random_state=28)

SVM_model.fit(X_train,y_train)

y_pred = best_SVM_model.predict(X_test)

In [33]:
def evaluate_metrics(y_test, y_pred):
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)
    return accuracy, precision, recall, f1, conf_matrix

accuracy, precision, recall, f1, conf_matrix = evaluate_metrics(y_test, y_pred)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 0.8695
Precision: 0.84593837535014
Recall: 0.8
F1 Score: 0.8223281143635127
Confusion Matrix:
[[1135  110]
 [ 151  604]]
